 ## Assignment 3

 This assignment is aimed to give you some practice using Jupyter Notebooks, R, and interpretting statistical output using real-world data. The notebook below will be used to generate the statistical output and you will also write up answers to specific questions related to the statistical output. The assignment will be guided and much of the R code will be provided for you, but specific aspects of the R code you will be asked to interact with and ultimately make a decision about appropriate values to include. The notebook should be run from the first code cell in sequential order, this means that you must run the beginning cells in order to be able to have access to the R packages needed for the assignment and that the data are read in appropriately.

 You may work in groups of up to 3 to complete the assignment. In these situations, please turn in one assignment in ICON with all group members nsong on the submission.

 *Assignment 3 Due*: **Monday, April 10th, by 11:59 pm**

## Description of the Data

These data are song data that were obtained from [Kaggle](https://www.kaggle.com/datasets/yasserh/song-popularity-dataset). In the data, each row is a single song and characteristics of those songs are depicted in the columns. The following is a brief summary of the attributes included in the data. 

+ **song_name**: The name of the song.
+ **song_popularity**: How popular the song was/is, 0 = not popular, 100 = very popular. 
+ **song_duration_ms**: How long the song is, in milliseconds.
+ **acousticness**: A score of how acoustic the song is, 0 = not acoustic, 1 = very acoustic.
+ **danceability**: How danceable the song is, 0 = not danceable, 1 = more danceable.
+ **energy**: The amount of energy the song conveys, 0 = little enery, 1 = a lot of energy.
+ **instrumentalness**: How instrumental the song is, 0 = not instrumental, 1 = instrumental.
+ **liveness**: The liveness of the song, 0 = not lively, 1 = very lively.
+ **loudness**: How loud the song is, smaller values mean quieter, larger values mean louder.
+ **speechiness**: How speechy the song is, 0 = not very speechy, 1 = very speechy.
+ **tempo**: The tempo of the song, smaller values are slower temp, larger values are faster tempo.
+ **audio_valence**: How positive the song is, 0 = not positive, 1 = more positive.
+ **danceability_group**: A categorical danceability attribute, low dance, medium dance, high dance.

Please don't hesitate to reach out with any data questions about the structure and interpretation of the variables in the data.

### Code Assistance

Furthermore, please reach out if there are specific questions regarding the code below. I do not want the code to be a barrier, but it does allow us to do more interesting things and work with more interesting data. We will provide any support to fix any errors you may encounter with the code. 

## Assignment Setup

**Run this cell first upon opening the notebook everytime** This cell loads the R packages and prepares the data for you.

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)
library(rpart)
library(rpart.plot)
library(rsample)

options(scipen = 999)

theme_set(theme_bw(base_size = 14))

song <- readr::read_csv("https://raw.githubusercontent.com/lebebr01/psqf_4143/main/data/song_data.csv") |>
    select(-key, -audio_mode, -time_signature) |> 
    mutate(danceability_group = cut_number(danceability, n = 3, 
    label = c('low dance', 'medium dance', 'high dance')))

set.seed(2023)
song_split <- initial_split(song, prop = .75)
song_train <- training(song_split)
song_test <- testing(song_split)

head(song_test)

## Question 1
 
For the remainder of the assignment, we'll explore the `song_popularity` attribute, which is a score representing how popular the song is/was. Pick another **continuous, quantitative** attribute to complete the code to create a scatterplot of the relationship between `song_popularity` and the attribute you pick. Add in the **continuous, quantitative** attribute of your choosing in place of "$$" below and add appropriate axis labels in place of "@@".

Examples of continuous outcomes to use in place of "$$" include: `energy`, `tempo`, `audio_valence`.

Note: The dashed line is forcing the relationship to be linear (a straight line) where as the solid light blue line allows the relationship to be flexible. 

In [ ]:
gf_point(song_popularity ~ $$, data = song_train, size = 3) |>
  gf_smooth(method = 'lm', linetype = 2, size = 1) |>
  gf_smooth(method = 'loess', linetype = 1, size = 1, color = 'lightblue') |>
  gf_labs(x = "@@",
          y = "@@")

## Question 2

Complete the code below that computes the correlations between the `song_popularity` attribute and a few other attributes. The output should show multiple correlations (one for each row) for each unique value of the `danceability_group` attribute. That is, this question is exploring if the relationship depicted in question 1 changes for songs that have low, medium, or high danceability. 

Use the same attribute from question #1 in place of "$$". You should have three unique correlations to interpret in the output below.

In [ ]:
song_train |>
  group_by(danceability_group) |>
  summarise(cor = cor(song_popularity ~ $$), 
            num_data = n()) |>
  filter(num_data > 10)

 ## Question 3
 Now fit a regression tree to the song training data to predict the popularity of the song. Complete the code below to add data attributes that you feel will help to predict the song popularity in place of "$$". Note, separate variables with a `+` symbol and include 1 attribute from each of the categories highlighted below.

 Select one attribute from *each* of the following categories. You should have a total of 2 attributes in the model.

 + **Instrument Type**: `instrumentalness`
 + **Song Energy**: `energy` or `tempo`

In [ ]:
song_pop <- rpart(song_popularity ~ $$, data = song_train, method = 'anova')

rpart.plot(song_pop, roundint = FALSE, type = 3, branch = .3)

## Question 4
 
The following code to generates the prediction accuracy on the withheld test data. The minimum, maximum, mean, median, standard deviation, and IQR are returned for you.

Note, the command below requires that the model from question #3 is fitted prior to running the code below. If an error occurs, please rerun all cells before this one. 

In [ ]:
song_test <- song_test |>
  mutate(pop_predicted = predict(song_pop, newdata = song_test),
         error = song_popularity - pop_predicted,
         absolute_error = abs(error))

song_test |>
  df_stats(~ absolute_error, min, max, mean, median, sd, IQR)

## Question 5

The following code generates conditional error for 6 ranges of song popularity. The resulting figure shows the minimum and maximum error shown by the horizontal line and the average error as a single point along the horizontal line.

*Note*, the command below requires that the model from question #3 and the code from question #4 are processed prior to running the code below. If an error occurs, please rerun all cells before this one. 

**Interpreting y-axis values**  
The values on the y-axis represent the range of song popularity for which the errors are computed for. That is, the y-axis value of (73,100] means that the data for this row of the figure include any songs that have a song popularity rating between 73 and 100. 

In [ ]:
cond_error <- song_test |>
  df_stats(absolute_error ~ cut_number(song_popularity, 6), mean, min, max) |>
  rename(song_pop = `cut_number(song_popularity, 6)`)  |>
  mutate(song_pop2 = gsub("]", "", str_split_fixed(as.character(song_pop), ",", n = 2)[,2]),
          song_pop_val = as.numeric(song_pop2),
          song_pop2 = forcats::fct_reorder(song_pop2, song_pop_val))

gf_pointrangeh(song_pop ~ mean + min + max, data = cond_error) |>
  gf_labs(y = "Song Popularity", 
                 x = "Absolute Error")

# Question 6

The following code fits a model that includes many more attributes to try to predict the song popularity. The same conditional error plot shown in question #5 is created for this new model with more attribute. The final regression tree output is also shown along with the descriptive absolute error statistics that can be compared with question #4.

*Note*, this code requires the code from questions #3 through #5 to be able to run the code below. 

In the figure, model 1 is the model from question 4, model 2 is the model from question 6. 

**Interpreting y-axis values**  
The values on the y-axis represent the range of song popularity for which the errors are computed for. That is, the y-axis value of (73,100] means that the data for this row of the figure include any songs that have a song popularity rating between 73 and 100. 

In [ ]:
song_pop_2 <- rpart(song_popularity ~ ., 
      data = select(song_train, -song_name), method = 'anova')

rpart.plot(song_pop_2, roundint = FALSE, type = 3, branch = .3)

song_test <- song_test |>
  mutate(pop_predicted_2 = predict(song_pop_2, newdata = song_test),
         error_2 = song_popularity - pop_predicted_2,
         absolute_error = abs(error_2))

song_test |>
  df_stats(~ absolute_error, min, max, mean, median, sd, IQR)

cond_error_2 <- song_test |>
  df_stats(absolute_error ~ cut_number(song_popularity, 6), mean, min, max) |>
  rename(song_pop = `cut_number(song_popularity, 6)`)  |> 
  mutate(song_pop2 = gsub("]", "", str_split_fixed(as.character(song_pop), ",", n = 2)[,2]),
          song_pop_val = as.numeric(song_pop2),
          song_pop2 = forcats::fct_reorder(song_pop2, song_pop_val),
         model = 'Model 2')

cond_error_comb <- dplyr::bind_rows(mutate(cond_error, model = "Model 1"), 
      cond_error_2)

gf_pointrangeh(song_pop ~ mean + min + max, data = cond_error_comb, 
     color = ~ model, position = position_dodge(width = .9)) |>
  gf_labs(y = "Song Popularity", 
                 x = "Absolute Error")